In [1]:
import gzip
import numpy
from collections import defaultdict
import math

In [2]:
def readGz(f):
  for l in gzip.open(f):
    yield eval(l)

In [3]:
data = list(readGz("train.json.gz"))
train_data = data
validation_data = data[len(data)/2:]

In [4]:
userRatings = defaultdict(dict)
itemRatings = defaultdict(dict)
beta_u = defaultdict(float)
beta_i = defaultdict(float)

In [5]:
for l in train_data:
    user,item = l['reviewerID'],l['itemID']
    userRatings[user][item] = l['rating']
    itemRatings[item][user] = l['rating']
    beta_u[user] = 0.0
    beta_i[item] = 0.0

In [6]:
def inner(x,y):
  return sum([x[i]*y[i] for i in range(len(x))])

In [7]:
def f(alpha, beta_u, beta_i, gamma_u, gamma_i, lamda, userRatings, itemRatings):
    obj = 0
    for u in userRatings:
        for i in userRatings[u]:
            obj += (alpha + beta_u + beta_i + inner(gamma_u[u], gamma_i[i]) - userRatings[u][i])**2
    regularizer = 0
    for u in userRatings:
        regularizer += beta_u**2 + inner(gamma_u[u], gamma_u[u])
    for i in itemRatings:
        regularizer += beta_i**2 + inner(gamma_i[i], gamma_i[i])
    obj += lamda * regularizer
    return obj

In [8]:
def fprime(alpha, beta_u, beta_i, gamma_u, gamma_i, lamda, userRatings, itemRatings, k):
    dobj = [0.0] * (1 + len(beta_u) + len(beta_i) + k * (len(gamma_u) + len(gamma_i)))


In [19]:
def iterative(lamda):
    alpha = 0
    for iteration in range(1000):
        original_alpha=alpha
        total = 0
        for u in userRatings:
            for i in userRatings[u]:
                temp = userRatings[u][i] - (beta_u[u] + beta_i[i])
                total += temp
        alpha = total/200000

        diff = abs(original_alpha-alpha)
        if diff < 1e-100:
            break
            
        for u in userRatings:
            total = 0
            for i in userRatings[u]:
                total += userRatings[u][i] - alpha - beta_i[i]
            beta_u[u] = total / (lamda + len(userRatings[u].keys()))

        for i in itemRatings:
            total = 0
            for u in itemRatings[i]:
                total += itemRatings[i][u] - alpha - beta_u[u]
            beta_i[i] = total/(lamda+len(itemRatings[i].keys()))

    return alpha, beta_u, beta_i

In [22]:
lamda = 5.45
alpha, beta_u, beta_i = iterative(lamda)

In [23]:
predictions = open("predictions_Rating.txt", 'w')
for l in open("pairs_Rating.txt"):
  if l.startswith("userID"):
    #header
    predictions.write(l)
    continue
  u,i = l.strip().split('-')
  result = alpha + beta_u[u] + beta_i[i]
  predictions.write(u + '-' + i + ',' + str(round(result,1)) + '\n')

predictions.close()